## Fine-tuning LLMs with HuggingFace, PEFT (LoRa/QLoRA)

---

<svg aria-roledescription="flowchart-v2" role="graphics-document document" viewBox="-8 -8 177.171875 470" style="max-width: 177.171875px;" xmlns="http://www.w3.org/2000/svg" width="100%" id="mermaid-svg-1750124783912-t0ns9iegq"><style>#mermaid-svg-1750124783912-t0ns9iegq{font-family:"trebuchet ms",verdana,arial,sans-serif;font-size:16px;fill:#f8f8f2;}#mermaid-svg-1750124783912-t0ns9iegq .error-icon{fill:#5a1d1d;}#mermaid-svg-1750124783912-t0ns9iegq .error-text{fill:#ff5555;stroke:#ff5555;}#mermaid-svg-1750124783912-t0ns9iegq .edge-thickness-normal{stroke-width:2px;}#mermaid-svg-1750124783912-t0ns9iegq .edge-thickness-thick{stroke-width:3.5px;}#mermaid-svg-1750124783912-t0ns9iegq .edge-pattern-solid{stroke-dasharray:0;}#mermaid-svg-1750124783912-t0ns9iegq .edge-pattern-dashed{stroke-dasharray:3;}#mermaid-svg-1750124783912-t0ns9iegq .edge-pattern-dotted{stroke-dasharray:2;}#mermaid-svg-1750124783912-t0ns9iegq .marker{fill:#f8f8f2;stroke:#f8f8f2;}#mermaid-svg-1750124783912-t0ns9iegq .marker.cross{stroke:#f8f8f2;}#mermaid-svg-1750124783912-t0ns9iegq svg{font-family:"trebuchet ms",verdana,arial,sans-serif;font-size:16px;}#mermaid-svg-1750124783912-t0ns9iegq .label{font-family:"trebuchet ms",verdana,arial,sans-serif;color:#f8f8f2;}#mermaid-svg-1750124783912-t0ns9iegq .cluster-label text{fill:#f8f8f2;}#mermaid-svg-1750124783912-t0ns9iegq .cluster-label span,#mermaid-svg-1750124783912-t0ns9iegq p{color:#f8f8f2;}#mermaid-svg-1750124783912-t0ns9iegq .label text,#mermaid-svg-1750124783912-t0ns9iegq span,#mermaid-svg-1750124783912-t0ns9iegq p{fill:#f8f8f2;color:#f8f8f2;}#mermaid-svg-1750124783912-t0ns9iegq .node rect,#mermaid-svg-1750124783912-t0ns9iegq .node circle,#mermaid-svg-1750124783912-t0ns9iegq .node ellipse,#mermaid-svg-1750124783912-t0ns9iegq .node polygon,#mermaid-svg-1750124783912-t0ns9iegq .node path{fill:#282a36;stroke:#191a21;stroke-width:1px;}#mermaid-svg-1750124783912-t0ns9iegq .flowchart-label text{text-anchor:middle;}#mermaid-svg-1750124783912-t0ns9iegq .node .label{text-align:center;}#mermaid-svg-1750124783912-t0ns9iegq .node.clickable{cursor:pointer;}#mermaid-svg-1750124783912-t0ns9iegq .arrowheadPath{fill:#d7d5c9;}#mermaid-svg-1750124783912-t0ns9iegq .edgePath .path{stroke:#f8f8f2;stroke-width:2.0px;}#mermaid-svg-1750124783912-t0ns9iegq .flowchart-link{stroke:#f8f8f2;fill:none;}#mermaid-svg-1750124783912-t0ns9iegq .edgeLabel{background-color:#282a3699;text-align:center;}#mermaid-svg-1750124783912-t0ns9iegq .edgeLabel rect{opacity:0.5;background-color:#282a3699;fill:#282a3699;}#mermaid-svg-1750124783912-t0ns9iegq .labelBkg{background-color:rgba(40, 42, 54, 0.5);}#mermaid-svg-1750124783912-t0ns9iegq .cluster rect{fill:rgba(68, 71, 90, 0.5);stroke:#6272a4;stroke-width:1px;}#mermaid-svg-1750124783912-t0ns9iegq .cluster text{fill:#f8f8f2;}#mermaid-svg-1750124783912-t0ns9iegq .cluster span,#mermaid-svg-1750124783912-t0ns9iegq p{color:#f8f8f2;}#mermaid-svg-1750124783912-t0ns9iegq div.mermaidTooltip{position:absolute;text-align:center;max-width:200px;padding:2px;font-family:"trebuchet ms",verdana,arial,sans-serif;font-size:12px;background:#44475a;border:1px solid #6272a4;border-radius:2px;pointer-events:none;z-index:100;}#mermaid-svg-1750124783912-t0ns9iegq .flowchartTitleText{text-anchor:middle;font-size:18px;fill:#f8f8f2;}#mermaid-svg-1750124783912-t0ns9iegq :root{--mermaid-font-family:"trebuchet ms",verdana,arial,sans-serif;}</style><g><marker orient="auto" markerHeight="12" markerWidth="12" markerUnits="userSpaceOnUse" refY="5" refX="6" viewBox="0 0 10 10" class="marker flowchart" id="mermaid-svg-1750124783912-t0ns9iegq_flowchart-pointEnd"><path style="stroke-width: 1; stroke-dasharray: 1, 0;" class="arrowMarkerPath" d="M 0 0 L 10 5 L 0 10 z"/></marker><marker orient="auto" markerHeight="12" markerWidth="12" markerUnits="userSpaceOnUse" refY="5" refX="4.5" viewBox="0 0 10 10" class="marker flowchart" id="mermaid-svg-1750124783912-t0ns9iegq_flowchart-pointStart"><path style="stroke-width: 1; stroke-dasharray: 1, 0;" class="arrowMarkerPath" d="M 0 5 L 10 10 L 10 0 z"/></marker><marker orient="auto" markerHeight="11" markerWidth="11" markerUnits="userSpaceOnUse" refY="5" refX="11" viewBox="0 0 10 10" class="marker flowchart" id="mermaid-svg-1750124783912-t0ns9iegq_flowchart-circleEnd"><circle style="stroke-width: 1; stroke-dasharray: 1, 0;" class="arrowMarkerPath" r="5" cy="5" cx="5"/></marker><marker orient="auto" markerHeight="11" markerWidth="11" markerUnits="userSpaceOnUse" refY="5" refX="-1" viewBox="0 0 10 10" class="marker flowchart" id="mermaid-svg-1750124783912-t0ns9iegq_flowchart-circleStart"><circle style="stroke-width: 1; stroke-dasharray: 1, 0;" class="arrowMarkerPath" r="5" cy="5" cx="5"/></marker><marker orient="auto" markerHeight="11" markerWidth="11" markerUnits="userSpaceOnUse" refY="5.2" refX="12" viewBox="0 0 11 11" class="marker cross flowchart" id="mermaid-svg-1750124783912-t0ns9iegq_flowchart-crossEnd"><path style="stroke-width: 2; stroke-dasharray: 1, 0;" class="arrowMarkerPath" d="M 1,1 l 9,9 M 10,1 l -9,9"/></marker><marker orient="auto" markerHeight="11" markerWidth="11" markerUnits="userSpaceOnUse" refY="5.2" refX="-1" viewBox="0 0 11 11" class="marker cross flowchart" id="mermaid-svg-1750124783912-t0ns9iegq_flowchart-crossStart"><path style="stroke-width: 2; stroke-dasharray: 1, 0;" class="arrowMarkerPath" d="M 1,1 l 9,9 M 10,1 l -9,9"/></marker><g class="root"><g class="clusters"/><g class="edgePaths"><path marker-end="url(#mermaid-svg-1750124783912-t0ns9iegq_flowchart-pointEnd)" style="fill:none;" class="edge-thickness-normal edge-pattern-solid flowchart-link LS-A LE-B" id="L-A-B-0" d="M80.586,34L80.586,38.167C80.586,42.333,80.586,50.667,80.586,58.117C80.586,65.567,80.586,72.133,80.586,75.417L80.586,78.7"/><path marker-end="url(#mermaid-svg-1750124783912-t0ns9iegq_flowchart-pointEnd)" style="fill:none;" class="edge-thickness-normal edge-pattern-solid flowchart-link LS-B LE-C" id="L-B-C-0" d="M80.586,118L80.586,122.167C80.586,126.333,80.586,134.667,80.586,142.117C80.586,149.567,80.586,156.133,80.586,159.417L80.586,162.7"/><path marker-end="url(#mermaid-svg-1750124783912-t0ns9iegq_flowchart-pointEnd)" style="fill:none;" class="edge-thickness-normal edge-pattern-solid flowchart-link LS-C LE-D" id="L-C-D-0" d="M80.586,202L80.586,206.167C80.586,210.333,80.586,218.667,80.586,226.117C80.586,233.567,80.586,240.133,80.586,243.417L80.586,246.7"/><path marker-end="url(#mermaid-svg-1750124783912-t0ns9iegq_flowchart-pointEnd)" style="fill:none;" class="edge-thickness-normal edge-pattern-solid flowchart-link LS-D LE-E" id="L-D-E-0" d="M80.586,286L80.586,290.167C80.586,294.333,80.586,302.667,80.586,310.117C80.586,317.567,80.586,324.133,80.586,327.417L80.586,330.7"/><path marker-end="url(#mermaid-svg-1750124783912-t0ns9iegq_flowchart-pointEnd)" style="fill:none;" class="edge-thickness-normal edge-pattern-solid flowchart-link LS-E LE-F" id="L-E-F-0" d="M80.586,370L80.586,374.167C80.586,378.333,80.586,386.667,80.586,394.117C80.586,401.567,80.586,408.133,80.586,411.417L80.586,414.7"/></g><g class="edgeLabels"><g class="edgeLabel"><g transform="translate(0, 0)" class="label"><foreignObject height="0" width="0"><div style="display: inline-block; white-space: nowrap;" xmlns="http://www.w3.org/1999/xhtml"><span class="edgeLabel"></span></div></foreignObject></g></g><g class="edgeLabel"><g transform="translate(0, 0)" class="label"><foreignObject height="0" width="0"><div style="display: inline-block; white-space: nowrap;" xmlns="http://www.w3.org/1999/xhtml"><span class="edgeLabel"></span></div></foreignObject></g></g><g class="edgeLabel"><g transform="translate(0, 0)" class="label"><foreignObject height="0" width="0"><div style="display: inline-block; white-space: nowrap;" xmlns="http://www.w3.org/1999/xhtml"><span class="edgeLabel"></span></div></foreignObject></g></g><g class="edgeLabel"><g transform="translate(0, 0)" class="label"><foreignObject height="0" width="0"><div style="display: inline-block; white-space: nowrap;" xmlns="http://www.w3.org/1999/xhtml"><span class="edgeLabel"></span></div></foreignObject></g></g><g class="edgeLabel"><g transform="translate(0, 0)" class="label"><foreignObject height="0" width="0"><div style="display: inline-block; white-space: nowrap;" xmlns="http://www.w3.org/1999/xhtml"><span class="edgeLabel"></span></div></foreignObject></g></g></g><g class="nodes"><g transform="translate(80.5859375, 17)" id="flowchart-A-32" class="node default default flowchart-label"><rect height="34" width="118.140625" y="-17" x="-59.0703125" ry="0" rx="0" style="fill:#f9f;stroke:#333;stroke-width:2px;" class="basic label-container"/><g transform="translate(-51.5703125, -9.5)" style="" class="label"><rect/><foreignObject height="19" width="103.140625"><div style="display: inline-block; white-space: nowrap;" xmlns="http://www.w3.org/1999/xhtml"><span class="nodeLabel">Original Model</span></div></foreignObject></g></g><g transform="translate(80.5859375, 101)" id="flowchart-B-33" class="node default default flowchart-label"><rect height="34" width="145.75" y="-17" x="-72.875" ry="0" rx="0" style="fill:#bbf;stroke:#333;stroke-width:2px;" class="basic label-container"/><g transform="translate(-65.375, -9.5)" style="" class="label"><rect/><foreignObject height="19" width="130.75"><div style="display: inline-block; white-space: nowrap;" xmlns="http://www.w3.org/1999/xhtml"><span class="nodeLabel">8-bit Quantization</span></div></foreignObject></g></g><g transform="translate(80.5859375, 185)" id="flowchart-C-35" class="node default default flowchart-label"><rect height="34" width="161.171875" y="-17" x="-80.5859375" ry="0" rx="0" style="fill:#bbf;stroke:#333;stroke-width:2px;" class="basic label-container"/><g transform="translate(-73.0859375, -9.5)" style="" class="label"><rect/><foreignObject height="19" width="146.171875"><div style="display: inline-block; white-space: nowrap;" xmlns="http://www.w3.org/1999/xhtml"><span class="nodeLabel">Double Quantization</span></div></foreignObject></g></g><g transform="translate(80.5859375, 269)" id="flowchart-D-37" class="node default default flowchart-label"><rect height="34" width="98.0625" y="-17" x="-49.03125" ry="0" rx="0" style="fill:#bbf;stroke:#333;stroke-width:2px;" class="basic label-container"/><g transform="translate(-41.53125, -9.5)" style="" class="label"><rect/><foreignObject height="19" width="83.0625"><div style="display: inline-block; white-space: nowrap;" xmlns="http://www.w3.org/1999/xhtml"><span class="nodeLabel">NF8 Format</span></div></foreignObject></g></g><g transform="translate(80.5859375, 353)" id="flowchart-E-39" class="node default default flowchart-label"><rect height="34" width="94.015625" y="-17" x="-47.0078125" ry="0" rx="0" style="fill:#bfb;stroke:#333;stroke-width:2px;" class="basic label-container"/><g transform="translate(-39.5078125, -9.5)" style="" class="label"><rect/><foreignObject height="19" width="79.015625"><div style="display: inline-block; white-space: nowrap;" xmlns="http://www.w3.org/1999/xhtml"><span class="nodeLabel">PEFT/LoRA</span></div></foreignObject></g></g><g transform="translate(80.5859375, 437)" id="flowchart-F-41" class="node default default flowchart-label"><rect height="34" width="139.890625" y="-17" x="-69.9453125" ry="0" rx="0" style="fill:#f9f;stroke:#333;stroke-width:2px;" class="basic label-container"/><g transform="translate(-62.4453125, -9.5)" style="" class="label"><rect/><foreignObject height="19" width="124.890625"><div style="display: inline-block; white-space: nowrap;" xmlns="http://www.w3.org/1999/xhtml"><span class="nodeLabel">Fine-tuned Model</span></div></foreignObject></g></g></g></g></g></svg>

### 0 - Setup

In [21]:
# %pip install transformers==4.40.2, peft==0.10.0
# %pip install torch==2.2.1+cu121 torchvision==0.17.1+cu121 torchaudio==2.2.1+cu121 --index-url https://download.pytorch.org/whl/cu121
# %pip install accelerate==1.7.0
# %pip install bitsandbytes==0.41.1 --prefer-binary --extra-index-url=https://jllllll.github.io/bitsandbytes-windows-webui
# %pip install bitsandbytes-cuda117==0.26.0.post2
# %pip install -i https://pypi.org/simple/ bitsandbytes
# %pip install trl

In [22]:
# HuggingFace
import transformers, peft
from datasets import load_dataset
from peft import LoraConfig, AutoPeftModelForCausalLM, prepare_model_for_kbit_training, get_peft_model
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    DataCollatorForLanguageModeling
)
from trl import SFTTrainer
import bitsandbytes as bnb

# Models/ML
from ollama import chat
import torch

# System
from dotenv import load_dotenv
import os, sys, subprocess
import gc # Garbage collector

# Extras
import accelerate
from importlib.metadata import version
import warnings
from tqdm import tqdm # Progress bar
# warnings.filterwarnings('ignore')

# Model and dataset configuration
# model_name = "meta-llama/Llama-2-7b-hf"
model_name = 'TinyLlama/TinyLlama-1.1B-Chat-v1.0' # More suitable for my GPU
train_dataset_path = "../data/training_dataset.jsonl"
output_model = '../models/TinyLlama-1.1b-Chat-FineTuned-v1.0'

# Loading environment variables
load_dotenv()
HF_TOKEN = os.getenv('HUGGINGFACE_TOKEN')

Asserting that transformers and peft lib versions are compatible

In [23]:
print('transformers version:', transformers.__version__)
#print('peft version:', peft.__version__)
print('bitsandbytes version:', version('bitsandbytes'))
print('trl version:', version('trl'))
print('accelerate version:', accelerate.__version__)
print(f"PyTorch version: {torch.__version__}. - Must be a version with GPU (CUDA) support, not CPU only.")

# Asserting versions
# assert transformers.__version__ == '4.40.2', 'transformers version mismatch'
#assert peft.__version__ == '0.10.0', 'peft version mismatch'

transformers version: 4.52.4
bitsandbytes version: 0.46.0
trl version: 0.18.2
accelerate version: 1.7.0
PyTorch version: 2.2.1+cu121. - Must be a version with GPU (CUDA) support, not CPU only.


In [24]:
def check_nvidia_smi():
    try:
        result = subprocess.run(['nvidia-smi'], capture_output=True, text=True)
        return result.stdout
    except FileNotFoundError:
        return "nvidia-smi not found. NVIDIA drivers may not be installed."

print("=== NVIDIA Driver Check ===")
print(check_nvidia_smi())

=== NVIDIA Driver Check ===
Fri Jun 20 22:30:01 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 576.57                 Driver Version: 576.57         CUDA Version: 12.9     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce GTX 1050      WDDM  |   00000000:01:00.0 Off |                  N/A |
| N/A   41C    P8            N/A  / 5001W |    3999MiB /   4096MiB |      0%      Default |
|                                         |                        |                  N/A |
+-------------------

In [25]:
# Get GPU memory info
gpu_memory = torch.cuda.get_device_properties(0).total_memory
max_memory = {0: f"{gpu_memory * 0.85 / 1e9:.1f} GB"}  # Use 85% of GPU memory

print(f"GPU Memory: {gpu_memory / 1e9:.1f} GB")
print(f"Max memory for model: {max_memory}")

GPU Memory: 4.3 GB
Max memory for model: {0: '3.4 GB'}


CUDA Availability Check

In [26]:
# Check CUDA availability
use_gpu = torch.cuda.is_available()
print('CUDA Available:', use_gpu)
print('Current CUDA device:', torch.cuda.current_device() if use_gpu else 'No CUDA device')
print('CUDA version:', torch.version.cuda if use_gpu else 'Not Available')

if use_gpu:
    print(f'CUDA device: {torch.cuda.get_device_name(0)}')
    print(f'CUDA capability:', torch.cuda.get_device_capability(0))

CUDA Available: True
Current CUDA device: 0
CUDA version: 12.1
CUDA device: NVIDIA GeForce GTX 1050
CUDA capability: (6, 1)


#### Releasing GPU Memory

In [27]:
torch.cuda.empty_cache()
print('Bytes collected:')
gc.collect()

Bytes collected:


10115

---

### 1- Loading Train Data

In [28]:
# Training data for Fine-Tuning
dataset = load_dataset("json", data_files=train_dataset_path, split="train")


# Formatting the dataset for training
def formatting(example):
    return {"text": f"### Prompt:\n{example['prompt']}\n\n### Response:\n{example['response']}"}

dataset = dataset.map(formatting)



### 1.1- Loading Tokenizer and transforming Train Data

In [29]:
# Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

# Tokenize train dataset
def preprocess(ex):
    text = ex["prompt"] + ex["response"]
    tokenized = tokenizer(text, truncation=True, padding="max_length", max_length=512)
    # Creating 'labels' column for the Trainer
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

tokenized = dataset.map(preprocess, batched=False)

---

### 2- Loading and Configuring Model

In [34]:
# Configure 4-bit quantization for efficient GPU usage
bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,  # Enable 8-bit quantization
    llm_int8_threshold=6.0,  # Threshold for outlier detection
    llm_int8_has_fp16_weight=True,  # Use fp16 for weights
    llm_int8_enable_fp32_cpu_offload=True # Offload to CPU
)

# Load Model with quantization
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True,
    use_cache=False,  # Important for training
    low_cpu_mem_usage=True,
)

OutOfMemoryError: CUDA out of memory. Tried to allocate 22.00 MiB. GPU 0 has a total capacity of 4.00 GiB of which 0 bytes is free. Of the allocated memory 10.32 GiB is allocated by PyTorch, and 659.13 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

---

### 3- Preparing for PEFT - Applying LoRA/QLoRA

In [96]:
# Prepare for PEFT
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

# LoRA Config
lora_config = LoraConfig(
    r=8, # Rank of the LoRA matrix, the number of trainable parameters (the higher the more trainable parameters, but also more memory and computation)
    lora_alpha=32, # Scaling factor for the LoRA matrix
    target_modules=["q_proj", "v_proj"], # Query and Value projection layers for TinyLlama
    lora_dropout=0.05, # Dropout rate for the LoRA matrix
    bias="none", # Bias for the LoRA matrix (not used for TinyLlama)
    task_type="CAUSAL_LM" # Task type for the LoRA matrix
)

model = get_peft_model(model, lora_config)

### Trainable Parameters Overview

In [97]:
print(model.print_trainable_parameters())

trainable params: 1,126,400 || all params: 1,101,174,784 || trainable%: 0.10229075496156657
None


---

### 4- Data Collator

A data collator is a crucial component in the training pipeline that prepares batches of data for the model. 
In this case, we're using DataCollatorForLanguageModeling which:
1. Pads sequences to the same length within each batch
2. Creates attention masks to handle the padding
3. Prepares the labels for language modeling

The mlm=False parameter indicates we're doing causal language modeling (predicting next token) 
rather than masked language modeling (predicting masked tokens).

This collator is necessary because:
- It ensures all sequences in a batch have the same length through padding
- It properly formats the input for the model's forward pass
- It handles the creation of labels for the language modeling task
- It optimizes memory usage by padding only within each batch rather than to a fixed length -->


In [98]:
data_collator = DataCollatorForLanguageModeling(tokenizer, 
                                                mlm=False) # As we are working with Causal Language Modeling (predicting next token)

---

### 5- Configure Training Arguments

I'll  avoid Trainer, its returning error when trying to train():

"The safest path on your setup is to avoid Trainer and instead train using a custom training loop with Accelerate, which gives you more control and avoids hidden offloading. Sometimes Hugging Face's Trainer tries to offload to CPU automatically if it detects low VRAM."

In [99]:
# Training Arguments
training_args = TrainingArguments(
    output_dir=output_model,
    per_device_train_batch_size=16,
    gradient_accumulation_steps=4,
    optim="paged_adamw_32bit",
    learning_rate=2e-4,
    lr_scheduler_type="cosine",
    save_strategy="epoch",
    logging_steps=10,
    num_train_epochs=3,
    max_steps=250,
    fp16=True
)

# Trainer
trainer = SFTTrainer(
        model=model,
        train_dataset=tokenized,
        peft_config=lora_config,
        args=training_args,
        data_collator=data_collator
    )

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


---

### 6 - Training

In [100]:
trainer.train()

RuntimeError: expected mat1 and mat2 to have the same dtype, but got: struct c10::Half != signed char

---

### 7- Saving Fine-Tuned LLM